# Bayesian Statistics and Probabilistic Programming

This ~1 hour talk is designed to give an introduction to Bayesian Statistics and probabilistic programming with PyMC. Since there is a large body of good resources for learning these topics, the goal of this notebook is to give an introduction to this area in the context of problems we might encounter at zData. In doing so we will also cover concepts that have broad generality and are valuable in a wide variety of circumstances.

This talk is meant to be presented in a notebook form. The idea is that the notebook can then be disseminated, so that attendees both have notes from the talk and also code that can form the scaffold for some probabilistic programming of their own.

## Motivation - when to use Bayesian / statistical approaches instead of machine learning

Let’s start by identifying what characteristics of problems we might encounter at zData lend themselves to a statistical approach as opposed to a machine learning approach.

We typically want to use statistical approaches when:
- We want to determine probabilities of things happening, as opposed to just making har predictions without probabilities.
- The dimensionality of the input space is small and/or there are lots of duplicate examples.

The story is more nuanced than this, but I think these are good starting questions that usually give a clear answer on what sort of path to take in tackling the problem.

### Example Problem #1

The classic coin flip problem. You’re given a rigged coin and given some number of flips, determine the probability of the next toss being heads or tails. TODO: Add image
- This fits (A) since our primary goal is to estimate a probability.
- It fits (B) since each example is just a binary heads/tails. There is no rich feature space a neural network can learn from. An alternative way of looking at (B) is that it's dealing with ‘counting’ problems. When we are dealing with counts of things, the input feature space is small, but there could be very many of those things. Such characteristics of our input data should act as a cue in your mind that a statistical approach may be appropriate.

### Example Problem #2

This is a slightly simplified version of a real-world zData problem (from The Italian Job). We want to predict the probability of a given alarm going off on a device in the next 3 days. We have a record of when alarms have gone off in the past. There are 10 kinds of alarms and hundreds of thousands of past alarm occurrences.

This example satisfies (A) since we want a probability, and it satisfies (B) because there are no features from a past alarm (aside from timestamp) to learn from, and there are many of them. But they are basically all the same.

How would a machine learning approach solve this? It’s difficult because the examples have no real features to learn from. You could frame each alarm occurrence as time since the last alarm in a neural network, but it’s kind of shoehorned and hard to get good probabilities out of a neural network anyway. Neural networks are more appropriate for taking rich ‘perceptual’ input spaces. Also, by default they don’t generate well-calibrated probabilities anyway, even if you’re using a sigmoid or softmax layer.

The actual problem stated above is more interesting still: There are actually hundreds of thousands of devices. On average each device only ever sees one alarm, and so data is very scarce on a per-device basis. However, each device is unique and behaves differently, so it is insufficient to pool all the data and make one probability estimate. We’ll touch on this approach more when we discuss hierarchical models.

But despite the problem being more complicated, the essence is still there: we want probabilities of events based on counting.

So when is statistics not appropriate? TODO: Add table of conditions. Computer Vision is a great example. For that problem we have a very high dimensional input space. Each image is unique, and there are limited senses in which you can count things and produce an effective statistical model. Language is also mostly best addressed with neural networks these days. However, there is a very rich history of statistical methods in natural language processing. Right before the NN revolution non-parametric Bayesian statistics approaches were the cool kid in town. They still probably have their place in niche applications.

What I’ve just said should not be taken to mean there is a clear dichotomy between statistical methods and machine learning methods. There are hybrid approaches, for example:
Neural networks that estimate the parameters of a statistical model. Slawek Smyl’s time series model that won the M4 time series prediction competition took this approach.
Probabilistic graphical models, which are basically a marriage of machine learning and statistics. Examples of probabilistic graphical models include things like conditional random fields, hidden Markov models and Bayesian Networks.


## What is a statistic?

Before we dive into doing some statistical inference, let’s just review some basics to highlight what statistics even is.

Wordnik gives a reasonable definition:
> A calculated numerical value (such as the sample mean) that characterizes some aspect of a sample set of data, and that is often meant to estimate the true value of a corresponding parameter (such as the population mean) in an underlying population.

They’re numbers that characterize data.

To illustrate the difference between statistical and machine learning approaches, we can consider the difference between statistical language models and neural network based language models.

Statistical language models used to be the most popular form of language model. The basic idea was to express the probability a word occurring given the sequence of preceding words by counting the amount if times that word occurs in that context.

For example, P(mat | the cat sat on the) - the probability that ‘mat’ follows the sequence ‘the cat sat on the mat’. We can determine this probability by counting how many times ‘the cat sat on the’ appears in the training data, and observing what proportion are followed by ‘mat’. If ⅘ instances end with mat, our probability estimate is 0.8. We determined this probability using statistics.

Contrast this with neural language models. We create representations of words in some vector space, do a bunch of transformations and arrive at a predicted next word. Here we don’t do counting, and we don’t make well-calibrated probability distributions, even though we typically will have valid probabilities (numbers that lie between 0 and 1 and sum to 1). We instead follow the backprop algorithm and learn neural network parameters. It’s fair to say these parameters ‘characterize the data’ and could be considered statistics, but we typically mean counting when we say statistics.

Note that neural LMs were introduced by Yoshua Bengio in 2003. It was only over decade later that they were superseded by neural LMs. Statistical LMs do well.

Statistics is all about methods to do these counts and use them to estimate probability distributions. This is called statistical inference.

Another note on terminology regarding the word ‘inference’. Statistical inference refers to this process of learning the distribution. This is distinct from ML ‘inference’ which refers to making predictions given a trained model. Statistical inference is more analogous to ‘training’ in the sense that is the learning phase.


## What is Bayesian statistics?

You’ll hear people harp on about ‘Bayesian statistics’, or ‘Bayesian’ methods. There are many concepts that are correlated with and get bundled up with Bayesian methods, but what really counts is that it reveals our stance towards statistics as being based on our belief about the world given the information we have. That is, it reflects our knowledge of the world, rather than necessarily some absolute truth. We use probability distributions to describe all values we are uncertain of. Probabilities are our measure of uncertainty.

Coming back to the coin flip example, our goal was to estimate the probability of a coin flip yielding heads. This is a hidden ‘parameter’ of the coin. A coin could be 50/50, or maybe 55/45. If the coin is actually toast with butter, it might be 80/20! But since we don’t know these probabilities for sure, we actually want a probability distribution over the probabilities.

The below plot might reflect our belief in the coin flip parameter. We start thinking it's most likely to be 50/50, but we also think there's a good chance the coin is rigged, so we are not so sure.


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy.stats import beta
sns.set_theme(style="darkgrid")

a, b = 1, 1
xs = np.linspace(0,1, num=100)
plt.plot(xs, beta.pdf(xs, a, b))
plt.ylabel('Prior belief')
plt.xlabel('Coin parameter')

In [ ]:
a, b = 2, 2
xs = np.linspace(0,1, num=100)
plt.plot(xs, beta.pdf(xs, a, b))
plt.ylabel('Prior belief')
plt.xlabel('Coin parameter')

Perhaps some time later we see a bunch more heads than tails, our belief might be different:

In [ ]:
a, b = 8, 2
xs = np.linspace(0,1, num=100)
plt.plot(xs, beta.pdf(xs, a, b))
plt.ylabel('Prior belief')
plt.xlabel('Coin parameter')

The crucial point is that the coin parameter is still unknown, and so we give it a distribution. This distribution reflects our belief and it may change with observations from the real world, but it is still uncertain.

In [ ]:
import arviz as az
import matplotlib
import numpy as np
import pymc as pm

## Model 1 How to do it: A Bayesian coin flip example

So let's actually do some statistical inference now.

Our goal is to come up with a distribution that reflects our belief of the parameters of our model. Learning this parameter is the 'inference' bit. The inference will take into account what we believe to be true without having seen observations (our prior), as well as the observations.

In [ ]:
obs = np.array([True, True, True, True, False])
obs

In [ ]:
with pm.Model() as m1:
    p = pm.Beta('p', alpha=1, beta=1)
    likelihood = pm.Bernoulli('likelihood', p=p, observed=obs)

In [ ]:
pm.model_to_graphviz(m1)

In this diagram, grey refers to observed variables. White refers to hidden ('latent') variables. The rectangle represents duplication of observations. So we have 5 observed Bernoulli trials. These are parametrized by a probability that itself is drawn from a distribution. We don't know what the p is, but we have an idea of the distribution.

Another way of looking at this is our model tries to reconcile two pieces of information. Firstly we have a prior which reflects knowledge we have before observing the trials. Secondly we have some observations. The model describes the best way of reconciling these two pieces of information.

In [ ]:
with m1:
    trace = pm.sample(draws=1000, tune=1000)

In [ ]:
az.plot_trace(trace, var_names=['p'], combined=True)

So what is inference process learning here? It is finding some parameters that are consistent with our prior knowledge about the coin, and what we observe in our trials. We’ll delve into this a bit more.

Note the above example could have been solved analytically with some maths quite easily. But the point is that often the distributions we want to specify don’t have easy analytical solutions, and this is where probabilistic programming shines.


Or perhaps do a linear regression example too. Regardless, the take home message should be:
You get a probability distribution as an outcome
You basically needed to do no mathematics. Just pick the right form of a distribution, which amounts to just knowing the range of your data.



## Bayes Theorem

I introduced PyMC for the coin flip example before talking about the maths. The core idea is that we aim to learn a probability distribution $P(\theta|X)$ over our parameters $\theta$ given our data $X$. This goal is our 'posterior'. TODO: Make the variable names above consistent with what we present here.

We start with our 'prior' belief about theta $P(\theta)$, not conditioned on seeing any data. We also have a 'likelihood' that describes the probability of our observations $P(X|\theta)$.

By the axioms of probability we can derive a relationship between the posterior, likelihood, and prior:

$P(\theta, X) = P(X|\theta)P(\theta)$

Similarly, we have:

$P(\theta, X) = P(\theta|X)P(X)$

So we see that:

$P(\theta|X)P(X) = P(X|\theta)P(\theta)$

Dividing by $P(X)$ on both sides gives us Bayes’ Theorem:

$$P(\theta|X) = \frac{P(X|\theta)P(\theta)}{P(X)}$$

If you use the right forms of distributions you can actually plug in values appropriately here. Say a Beta distribution for our prior, a bernoulli distribution for our likelihood, and by re-expressing $P(X)$ and doing some mathematical jiggery-pokery evaluate this denominator and come up with your posterior. But it requires all sorts of burdonsome things like ensuring that the forms of the distributions are compatible in a way that lets you produce an analytical solution. Depending on the forms it can be totally intractible to compute $P(X)$. Actually, the coin flip example above has a fairly straightforward analytical solution, so MCMC is not needed. At some point here I need to mention Markov Chain Monte Carlo at least to give a head nod to what happens under the hood.

The magic of using probabilistic programming languages like PyMC is that you just define your prior and likelihood and then you can sample from the posterior directly without having to do the involved mathematical legwork.


## Model 2: Linear regression



## Model 3: Partial pooling / hierarchical models

The models up until now have been able to be computed analytically and are basically pretty simple models that don't fully demonstrate the abilities of Bayesian techniques.

Imagine you are a biologist, and you travel around observing animals in the wild. Stumble upon the nesting site of some wild Zebra finches:

![alt text](./zebra_finches.jpg)

There are 10 female finches. Your goal is to estimate for each finch the probability that eggs they lay hatch. You get to stay for a month, observing the birds and seeing the eggs hatch.

This may not seem like the most zData-relevant example, but you can replace finches with anything. The approach and method is zData-friendly.

Here is the data:


In [ ]:
import pandas as pd
data = pd.DataFrame.from_records([(0, 2), (45, 103), (58, 80), (21, 50), (65, 104),(42, 95)], columns=['hatched', 'eggs'])
data


### Solution 1: A single estimate for all finches.

The first option is to simply estimate the probability a finch egg will hatch and pool the data across all finches to estimate this probability. This approach is known as 'complete pooling'.

In [ ]:
sum(data['hatched'])

In [ ]:
sum(data['hatched'])/sum(data['eggs'])

The above gives us a single value, a 'point estimate' of egg hatching probability, produced via maximum likelihood estimation. We can also do a Bayesian approach, where we estimate a distribution. Using our model earlier.

Note here that we can also use a Binomial distribution.

In [ ]:
with pm.Model() as m2:
    p = pm.Beta('p', alpha=1, beta=1)
    y = pm.Binomial('y', p=p, n=sum(data['eggs']), observed=sum(data['hatched']))


In [ ]:
pm.model_to_graphviz(m2)

In [ ]:
with m2:
    trace2 = pm.sample(draws=1000, tune=1000)

In [ ]:
az.plot_trace(trace2, var_names=['p'], combined=True)

Here we see a distribution over possible egg hatching rates. Our model is Bayesian: we are incorporating prior beliefs and producing a dsitribution over possible values, rather than just estimating one.

But there is a problem! Each finch is different. Some are fertile and/or their mates are fertile and most of their eggs hatch. Others have low egg hatching rates.

Easy.

### Solution 2: Estimate per-finch parameters.

We can just make a model for each finch and estimate their egg hatching rate. Here's out maximum likelihood estimate

In [ ]:
data['hatched']/data['eggs']

We can also do the Bayesian version on a per finch basis

In [ ]:
finch_i = 0
with pm.Model() as m3:
    p = pm.Beta('p', alpha=1, beta=1)
    l = pm.Binomial('likelihood', p=p, n=data.iloc[finch_i]['eggs'], observed=data.iloc[finch_i]['hatched'])

In [ ]:
pm.model_to_graphviz(m3)

In [ ]:
with m3:
  trace3 = pm.sample(draws=1000, tune=1000)

In [ ]:
az.plot_trace(trace3, combined=True)

We can estimate per-finch parameters independently by making our parameters multidimensional:

In [ ]:
with pm.Model() as m4:
    p = pm.Beta('p', alpha=1, beta=1, shape=len(data))
    y = pm.Binomial("y", n=data['eggs'], p=p, shape=len(data), observed=data['hatched'])

Here our Beta prior is multidimensional, but the posterior of each parameter is estimated independently.

In [ ]:
pm.model_to_graphviz(m4)

In [ ]:
with m4:
    trace4 = pm.sample(draws=1000, tune=1000)
az.plot_trace(trace4, var_names=['p'], combined=True)

This diagram says that we have a bunch of random variables, 6 different probability variables, with a Beta prior. For each of those we find a posterior.

Note: Need to talk about the sampling over time diagram on the right.

But there's problems with this:

We throw out all the data about other finches when we make this estimate. Actually, in any given batch of eggs from a finch, the number of eggs that could hatch are highly variable. This finch seemed really fertile, but it might have just been lucky and it is not innately more fertile.

In the first model we treated all finches as the same and had no room for individuality. In the second model every finch was completely new to the model. The model had no memory of past finches and had to learn from scratch. But finches are finches, they're similar and we should draw on this information.

Enter partial pooling / hierarchical models. We give each finch it's own parameter, but the Finches share a common prior.

In [ ]:
with pm.Model() as m5:
    phi = pm.Beta("phi", alpha=1, beta=1)
    #kappa_log = pm.Exponential("kappa_log", lam=1.5)
    #kappa = pm.Deterministic("kappa", pm.math.exp(kappa_log))
    kappa_log = pm.HalfNormal('kappa_log', sigma=1)
    kappa = pm.Deterministic('kappa', pm.math.exp(kappa_log))
    #p = pm.Beta('p', alpha=phi * kappa, beta=(1.0 - phi) * kappa, dims='finch_ix')
    #p = pm.Beta('p', alpha=phi * kappa, beta=(1.0 - phi) * kappa, shape=len(data))
    p = pm.Beta('p', alpha=phi*kappa, beta=(1.0 - phi)*kappa, shape=len(data))
    y = pm.Binomial("y", n=data['eggs'], p=p, shape=len(data), observed=data['hatched'])

In [ ]:
pm.model_to_graphviz(m5)

In [ ]:
with m5:
    trace5 = pm.sample(draws=1000, tune=1000, target_accept=0.95)
az.plot_trace(trace5, var_names=['p', 'phi', 'kappa'], combined=True)

Other applications:
- Probability an event will happen on a device (The Italian Job)
- Election prediction (partial pooling where each pollster is an example)


## A taste of some other applications
Don’t step through in detail but show highlight and key takeaway of:
- hierarchical versions of these models that do partial pooling approaches.
- Non-parametric approaches where we don’t need the form of the distribution and/or the joint distributions are in a very big and complex space. E.g. joint word segmentation and alignment.
- Causal inference. Using several linear regression models and multiple regression to handle confounds and tease apart causal effects. Head-nod to causal DAGs and how there’s a really good framework for reasoning about causality and how it ties intimately into conditional independence and probabilistic graphical models.


## If we expand on this teaser
- How the sampling process (Markov Chain Monte Carlo) actually works
- More PyMC exercises
- Bayesian methods for neural networks
- Non-parametric Bayesian methods: where we don’t even need to know the shape of the distribution.
- How Causal inference works
- How to get distributions for deep learning predictions
- How to use dropout layers for confidence distribution for active learning datapoints [Lisa]


## Further Reading
Links to some further reading on stuff mentioned in the talk.
- Statistical Rethinking
- PyMC
